In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array

In [6]:
new_model = tf.keras.models.load_model('myCNNClassificationAwakeSleep.h5')

# アーキテクチャを確認
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0

In [9]:
##前処理系統
##kernelが止まるようなエラーがでる場合
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#データの読み込み
gsdata = pd.read_csv("./trainingdata.csv")
trainingImgData = np.zeros((len(gsdata["filenames"]), 128, 128, 1))
for i in range(0, len(gsdata["filenames"])):
    fnametmp = './image/%s' %gsdata["filenames"][i]
    trainingImgData[i] = np.array(img_to_array(load_img(fnametmp, grayscale=True, target_size=(128,128))))/255.0
trainingGSData = np.zeros((len(gsdata["abnormal"]),  1), dtype='int')
for i in range(0, len(gsdata["filenames"])):
   trainingGSData[i][0]=gsdata["abnormal"][i]
trainingGS = trainingGSData.copy()
from tensorflow.python.keras.utils import to_categorical
onehotlabel_train = to_categorical(trainingGS, 2)
results = list(new_model.predict_classes(trainingImgData, verbose=1))


398/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix

# 認識率（精度）を計算して表示．ラベルデータはone-hotじゃないよ！
score = accuracy_score(trainingGS, results)
print(score)

#混合行列を作成して表示．ラベルデータはone-hotじゃないよ！
cmatrix = confusion_matrix(trainingGS, results)
print(cmatrix)

0.9321608040201005
[[199   0]
 [ 27 172]]


In [13]:
#画像データを読み込みます．
testdata = pd.read_csv("./testdata.csv")
testImgData = np.zeros((len(testdata["filenames"]), 128, 128, 1))

for i in range(0, len(testdata["filenames"])):
    fnametmp = './image/%s' %testdata["filenames"][i]
    testImgData[i] = np.array(img_to_array(load_img(fnametmp, grayscale=True, target_size=(128,128))))/255.0

#正解データを読み込みます．
testGSData = np.zeros((len(testdata["abnormal"]),  1), dtype='int')
for i in range(0, len(testdata["filenames"])):
   testGSData[i][0]=testdata["abnormal"][i]

#正解のラベルを作ります．
testGS = testGSData.copy()
testGS = np.ravel(testGS)

#モデルに画像を代入します．
valigender = new_model.predict(testImgData)

In [14]:
valigender

array([[1.00000000e+00, 4.28494492e-32],
       [1.00000000e+00, 1.19131745e-31],
       [1.00000000e+00, 9.47137216e-34],
       [1.00000000e+00, 2.55848536e-37],
       [1.00000000e+00, 2.60157270e-37],
       [1.00000000e+00, 7.92089495e-37],
       [1.00000000e+00, 1.14454311e-31],
       [1.00000000e+00, 1.04238175e-29],
       [1.00000000e+00, 6.51741614e-29],
       [1.00000000e+00, 1.48083059e-26],
       [1.78742096e-01, 8.21257889e-01],
       [2.56860442e-02, 9.74313915e-01],
       [7.09615707e-01, 2.90384233e-01],
       [9.73250985e-01, 2.67489925e-02],
       [9.99993205e-01, 6.77062508e-06],
       [9.99932289e-01, 6.76482086e-05],
       [9.99997735e-01, 2.25134636e-06],
       [1.00000000e+00, 8.73367156e-09],
       [1.00000000e+00, 1.82159431e-12],
       [1.00000000e+00, 1.55180491e-15]], dtype=float32)

In [62]:
#画像データを読み込みます．
# testdata = pd.read_csv("./realtime.csv")
# testImgData = np.zeros((len(testdata["filenames"]), 128, 128, 1))
testImgData = np.zeros((1, 128, 128, 1))

fnametmp = './image/%s' %'test11.jpg'
testImgData[0] = np.array(img_to_array(load_img(fnametmp, grayscale=True, target_size=(128,128))))/255.0

#モデルに画像を代入します．
valigender = new_model.predict(testImgData)

In [63]:
valigender

array([[4.5129450e-04, 9.9954873e-01]], dtype=float32)

In [64]:
if valigender[0][0] < 0.5:
    print("sleep")
else:
    print("awake")

sleep


In [61]:
import cv2

deviceid = 0  # it depends on the order of USB connection.
capture = cv2.VideoCapture(deviceid)

ret, frame = capture.read()
ret, frame = capture.read()
cv2.imwrite('./image/test11.jpg', frame)

True

In [1]:
from PIL import Image
 
image = Image.open("./image/test11.jpg")
 
image.rotate(30).save("rotate_30.png")
image.rotate(30, Image.NEAREST, True).save("rotate_30_expand.png")
image.rotate(-30).save("rotate_-30.png")

In [3]:
import PIL
print(PIL.PILLOW_VERSION)

8.0.1


/anaconda3/envs/experiment/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in a future release. Use __version__ instead.
  
